In [1]:
import os

from python_includes import rainflow as rf

import numpy as np

import pandas as pd

import itertools

In [2]:
def read_h5_file(file_path):
    
    return pd.HDFStore(file_path, mode='r')

In [3]:
def read_bar_stresses(file_path):
    
    model = read_h5_file(file_path=file_path)
    
    bar_stress_df = model.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/BAR')
    
    model.close()
    
    return bar_stress_df

In [4]:
def read_results_domain_info(file_path):
    
    model = read_h5_file(file_path=file_path)
    
    domains_info = model.get('/NASTRAN/RESULT/DOMAINS')
    
    model.close()
    
    return domains_info

In [5]:
def get_subcase_mapping_dict(df):
    
    df = df.loc[:, ['ID', 'SUBCASE']].copy()

    df.set_index('ID', inplace=True)

    return df.to_dict()

In [6]:
def get_template_df():
    
    data = list(itertools.zip_longest(factor_lst, cust_case_list, desc_lst))

    df = pd.DataFrame(data, columns=['Factor', 'Case', 'Description'])

    df['Index'] = np.arange(1, len(data)+1)

    df['Factor_comb'] = np.NAN

    df['Case_comb'] = np.NaN

    df.loc[3:11,['Factor_comb']] = comb_factor

    df.loc[3:11,['Case_comb']] = comb_case

    df = df[['Factor', 'Case', 'Factor_comb', 'Case_comb', 'Description', 'Index']]
    
    return df

In [7]:
def get_spectrum_df(**kwargs):
    
    stress_df = kwargs.get('stress_df')
    
    template_df = kwargs.get('template_df')
    
    eid = kwargs.get('eid')
    
    eid_stress = stress_df[stress_df.EID == eid]

    cols = ['EID', 'AX', 'DOMAIN_ID', 'Case']

    eid_stress.columns = cols

    indx_cols = ['Case']

    eid_stress = eid_stress.set_index(indx_cols)

    # eid_stress

    df3 = template_df.join(eid_stress, on=indx_cols)

    df3  = df3.set_index(['Index'])

    df3 = df3.drop(['DOMAIN_ID'], axis=1)

    # df3

    eid_stress_comb =  eid_stress.drop(columns=['EID', 'DOMAIN_ID'])

    # eid_stress_comb
    
    eid_stress_comb.columns = ['AX_COMB']

    # eid_stress_comb

    df3 = df3.join(eid_stress_comb, on=['Case_comb'])

    # df3

    df3.AX_COMB = df3.AX_COMB * df3.Factor_comb

    # df3

    df3.AX_COMB.fillna(0, inplace=True)

    df3['Sigma_FEM'] = df3.AX  + df3.AX_COMB

    return df3

In [8]:
def get_SN_df(spectrum_df):
    
    results = []

    for low, high, mult in rf.extract_cycles(spectrum_df.Sigma_FEM.tolist()):
    #     mean = 0.5 * (high+low)
    #     rng = high - low
        results.append((high, low))

    rain_table = pd.DataFrame(results, columns=['Max', 'Min'])

    smax_const = 6.895

    kt_const = 3.0

    rain_table['EID'] = spectrum_df.EID.unique()[0]
    
    rain_table['R'] = rain_table.Min / rain_table.Max
    
    rain_table['Kt_const'] = kt_const
    
    rain_table['Smax_const'] = smax_const

    rain_table['S_Max'] = rain_table.Max / rain_table.Smax_const

    rain_table['Kt_sigma'] = rain_table.S_Max * rain_table.Kt_const

    rain_table['Seq'] = rain_table.Kt_sigma * (1 - rain_table.R) ** 0.52

    rain_table['Nf'] = 10 ** (20.83 - 9.09 * np.log10(rain_table.Seq))

    rain_table['_1_Over_Nf'] = 1 / rain_table.Nf

    rain_table['Index'] = np.arange(1, len(results)+1)

    rain_table  = rain_table.set_index(['Index'])
    
    sum_1_over_nf = rain_table._1_Over_Nf.sum()
    
    rain_table['One_Over_Nf_sum_of_1_over_Nf'] = rain_table._1_Over_Nf / sum_1_over_nf
    
    rain_table['Life'] = 1/sum_1_over_nf
    
    s1 = pd.Series(dict((v,k) for k,v in spectrum_df.Sigma_FEM.iteritems()))
    # s1

    rain_table['max_line_Num'] = rain_table.Max.map(s1)
    
    rain_table['min_line_Num'] = rain_table.Min.map(s1)
    
    rain_table['pct_damage'] = rain_table.One_Over_Nf_sum_of_1_over_Nf * 100
    
    rain_table['pct_damage'] = rain_table['pct_damage'].map('{:,.1f}%'.format)
    
    rain_table = sn_reorder(rain_table)
    
    return rain_table

In [9]:
def sn_reorder(df):
    
    lst_order = ['EID', 'Max', 'Min', 'max_line_Num', 'min_line_Num', 'R', 'Kt_const', 'Smax_const',
                 'S_Max', 'Kt_sigma', 'Seq', 'Nf', '_1_Over_Nf', 'One_Over_Nf_sum_of_1_over_Nf',
                 'Life', 'pct_damage']
    
    df = df[lst_order].copy()
    
    return df

In [10]:
def get_bar_stress_df(h5_filepath):
    
    bar_stress_df = read_bar_stresses(h5_filepath) # Read BAR Element Stresses

    bar_stress_df = bar_stress_df[['EID', 'AX', 'DOMAIN_ID']]

    domain_results = read_results_domain_info(h5_filepath)

    subcase_mapping = get_subcase_mapping_dict(domain_results)

    bar_stress_df['SUBCASE'] = bar_stress_df.DOMAIN_ID.map(subcase_mapping['SUBCASE']).copy()
    
    return bar_stress_df

In [11]:
def get_premod_bar_element_life(**kwargs):
    
    PREMOD_BAR_STRESS_DF = kwargs.get('stress_df')
    
    TEMPLATE_DF = kwargs.get('template_df')
    
    ELEMENT_ID = kwargs.get('eid')

    try:
        
        premod_spectrum_df = get_spectrum_df(eid = ELEMENT_ID, template_df = TEMPLATE_DF, stress_df = PREMOD_BAR_STRESS_DF)

        premod_rain_table = get_SN_df(premod_spectrum_df)

        # PRE_MOD Life Prediction Value
        premod_life  = round(premod_rain_table.Nf.tolist()[-1], 1)

        spec_df = change_spectrum_data_format(premod_spectrum_df)
        
        write_spectrum_data_to_file(spec_df, ELEMENT_ID, 'PRE-MOD')
        
        sn_df = change_SN_data_format(premod_rain_table)
        
        write_SN_data_to_file(sn_df, ELEMENT_ID, 'PRE-MOD')
        
    except:
        
        print('Failed Writing Fatigue data for PREMOD element id: {}'.format(ELEMENT_ID))
        
        pass
        
    
    return premod_life

In [12]:
def get_postmod_bar_element_life(**kwargs):
    
    POSTMOD_BAR_STRESS_DF = kwargs.get('stress_df')
    
    TEMPLATE_DF = kwargs.get('template_df')
    
    ELEMENT_ID = kwargs.get('eid')
    
    try:
        
        postmod_spectrum_df = get_spectrum_df(eid = ELEMENT_ID, template_df = TEMPLATE_DF, stress_df = POSTMOD_BAR_STRESS_DF)

        postmod_rain_table = get_SN_df(postmod_spectrum_df)

        # POST_MOD Life Prediction Value
        postmod_life  = round(postmod_rain_table.Nf.tolist()[-1], 1)        
        
        spec_df = change_spectrum_data_format(postmod_spectrum_df)
        
        write_spectrum_data_to_file(spec_df, ELEMENT_ID, 'POST-MOD')
        
        sn_df = change_SN_data_format(postmod_rain_table)
        
        write_SN_data_to_file(sn_df, ELEMENT_ID, 'POST-MOD')
        
    except:
        
        print('Failed Writing Fatigue data for POSTMOD element id: {}'.format(ELEMENT_ID))
        
        pass    
    
    return postmod_life

# Change Spectrum data decimal format.

In [13]:
def change_spectrum_data_format(df):
    
    df.fillna(value=0, inplace=True)

    df[['Case', 'Case_comb']] = df[['Case', 'Case_comb']].astype(int)

    spec_decimals = pd.Series([2, 0, 4, 0, 0, 2, 2, 2], 
                         index=['Factor', 'Case', 'Factor_comb', 'Case_comb', 'EID', 'AX', 'AX_COMB', 'Sigma_FEM'])

    df = df.round(spec_decimals).copy()

    # df

    # df.dtypes

    change_spectrum_labels =  ['Factor', 'Case', 'Factor_comb', 'Case_comb', 'Description', 'EID', 'σFEM', 'σFEM_COMB', 'ΣσFEM']

    df.columns = change_spectrum_labels
    
    return df

# Change S-N data table decimal format.

In [14]:
def change_SN_decimal_format(df):
    
    sn_decimals = pd.Series([2, 2, 1, 3, 2, 3, 2, 2, 2, 0, 1], 
                         index=['Max',
                                'Min', 
                                'Mult', 
                                'R', 
                                'Kt_const', 
                                'Smax_const', 
                                'S_Max', 
                                'Kt_sigma', 
                                'Seq', 
                                'One_Over_Nf_sum_of_1_over_Nf',
                                'Life'])

    df = df.round(sn_decimals)
    
    
    change_SN_labels = ['EID', 'Max', 'Min', 'max_line_Num', 'min_line_Num', 'R', 'Kt_const', 'Smax_const', 
                        'S_Max', 'Kt·σ in MPa', 'Seq in MPa', 'Nf', '1/Nf', '1/(Nf·Σ(1/Nf))', 'Life', '% damage']
    
    df.columns = change_SN_labels
    
    return custom_SN_data(df)

In [15]:
def custom_SN_data(df):
    
    df = df[['EID', 'Max', 'Min', 'max_line_Num', 'min_line_Num', 'R', 'Kt_const', 'Smax_const', 
                        'S_Max', 'Kt·σ in MPa', 'Seq in MPa', 'Nf', '1/Nf', '1/(Nf·Σ(1/Nf))']]
    
    return df

In [16]:
def write_spectrum_data_to_file(spec_df, eid, resultType):
    
    with open('Fatigue_output.out','a', encoding="utf-8") as outfile:
        
        outfile.writelines(str(resultType) + ' Elm ' + str(eid) + '\n')
        
        spec_df.to_string(outfile, columns=spec_df.columns.tolist())
        
        outfile.writelines('\n \n \n')

In [17]:
def write_SN_data_to_file(sn_df, eid, resultType):
    
    with open('Fatigue_output.out','a', encoding="utf-8") as outfile:
        
        outfile.writelines(str(resultType) + ' Elm ' + str(eid) + '\n')
        
        sn_df.to_string(outfile, columns=sn_df.columns.tolist())
        
        outfile.writelines('\nΣ(1/Nf) ' + str(round(sn_df['1/Nf'].sum())))
        
        outfile.writelines('\n' + str(resultType) + ' life : \t' + str(round(sn_df.Nf.tolist()[-1], 1)) +'\t FC (unfactored)')
        
        outfile.writelines('\n \n \n')

# MAIN START

In [18]:
premod_h5 = 'G:/PROJECTS/CAEP1073 - EIS PC-12 Radar Installation/wip/FEM/GFEM- M3/M3.1run/1.pre-mod/sol101_fatigue/pc12-47-v1-6-fatigue.h5'

postmod_h5 = 'G:/PROJECTS/CAEP1073 - EIS PC-12 Radar Installation/wip/FEM/GFEM- M3/M3.1run/2.post-mod/sol101_fatigue/pc12-47-v1-6-fatigue.h5'

# Input Element ID
ELEMENT_ID = 1110329

# Custom loadcase list
cust_case_list = [502,
                    503,
                    503,
                    504,
                    503,
                    505,
                    503,
                    506,
                    503,
                    507,
                    503,
                    503,
                    508,
                    509,
                    508,
                    510,
                    508,
                    511,
                    508,
                    512,
                    508,
                    515,
                    516,
                    514,
                    502]


# Custom Description list
desc_lst =['Standing on the ground',
    'Cruise (1g Level Flight)',
    'Cruise (1g Level Flight) + Cabin Pressure',
    'Cruise with Vertical Gust vz = 10 [m/s] + Cabin Pressure',
    'Cruise (1g Level Flight) + Cabin Pressure',
    'Cruise with Vertical Gust vz = -10 [m/s] + Cabin Pressure',
    'Cruise (1g Level Flight) + Cabin Pressure',
    'Cruise with Lateral Gust vz = 10 [m/s] + Cabin Pressure',
    'Cruise (1g Level Flight) + Cabin Pressure',
    'Cruise with Lateral Gust vz = -10 [m/s] + Cabin Pressure',
    'Cruise (1g Level Flight) + Cabin Pressure',
    'Cruise (1g Level Flight)',
    'Final Approach (1g Level Flight)',
    'Final Approach with Vertical Gust vz = 10 [m/s]',
    'Final Approach (1g Level Flight)',
    'Final Approach with Vertical Gust vz = -10 [m/s]',
    'Final Approach (1g Level Flight)',
    'Final Approach with Lateral Gust vz = 10 [m/s]',
    'Final Approach (1g Level Flight)',
    'Final Approach with Lateral Gust vz = -10 [m/s]',
    'Final Approach (1g Level Flight)',
    'Landing (Spin Up) vz = 6 [ft/s]',
    'Landing (Spring Back) vz = 6 [ft/s]',
    'Braking nx = -1 [ - ]',
    'Standing on the Ground']

factor_lst = [1.0] * len(cust_case_list)

comb_factor  = 0.4482 # Factor to be multiplied with Pressure Case

comb_case = 523 # Pressure Case

In [19]:
TEMPLATE_DF = get_template_df()

# PREMOD OPERATIONS
PREMOD_BAR_STRESS_DF = get_bar_stress_df(premod_h5)

premod_spectrum_df = get_spectrum_df(eid = ELEMENT_ID, template_df = TEMPLATE_DF, stress_df = PREMOD_BAR_STRESS_DF)

premod_rain_table = get_SN_df(premod_spectrum_df)

# POSTMOD OPERATIONS
POSTMOD_BAR_STRESS_DF = get_bar_stress_df(postmod_h5)

postmod_spectrum_df = get_spectrum_df(eid = ELEMENT_ID, template_df = TEMPLATE_DF, stress_df = POSTMOD_BAR_STRESS_DF)

postmod_rain_table = get_SN_df(postmod_spectrum_df)

# bar_template = {
#     'template': TEMPLATE_DF,
#     'premod_bar_stress_df': PREMOD_BAR_STRESS_DF,
#     'postmod_bar_stress_df': POSTMOD_BAR_STRESS_DF
# }


pre_template = {
    'stress_df': PREMOD_BAR_STRESS_DF,
    'template_df': TEMPLATE_DF,
    'eid': ELEMENT_ID,
    'type': 'PRE-MOD'
    
}

post_template = {
    'stress_df': POSTMOD_BAR_STRESS_DF,
    'template_df': TEMPLATE_DF,
    'eid': ELEMENT_ID,
    'type': 'POST-MOD'
    
}

In [35]:
def custom_manipulations(**kwargs):
    
    STRESS_DF = kwargs.get('stress_df')
    
    TEMPLATE_DF = kwargs.get('template_df')
    
    ELEMENT_ID = kwargs.get('eid')
    
    TYPE = kwargs.get('type')

    try:
        
        spectrum_df = get_spectrum_df(eid = ELEMENT_ID, template_df = TEMPLATE_DF, stress_df = STRESS_DF)

        SN_df = get_SN_df(spectrum_df)

#         spec_df = change_spectrum_data_format(spectrum_df)
        
#         write_spectrum_data_to_file(spec_df, ELEMENT_ID, TYPE)
        
#         sn_df = change_SN_decimal_format(SN_df)
        
#         write_SN_data_to_file(sn_df, ELEMENT_ID, TYPE)
        
    except:
        
        print('Failed Writing Fatigue data for ' + TYPE + ' element id: {}'.format(ELEMENT_ID))
        
        pass
        
    df = output2_data_df(SN_df)
    
    return df

In [21]:
def output2_data_df(df):
    try:
        pre = df.loc[df.One_Over_Nf_sum_of_1_over_Nf == df.One_Over_Nf_sum_of_1_over_Nf.max(), :]
        return pre
    except:
        return pd.DataFrame({'A' : []})

In [22]:
# df = custom_manipulations(**pre_template)
# premod_rain_table

In [36]:
try:
    os.remove("Fatigue_output.out")
    print('Found: Fatigue_output.out \n Action: Deleted')
except FileNotFoundError:
    #print('File Not Exist')
    pass

premod_bar_eids = PREMOD_BAR_STRESS_DF.EID.unique()

postmod_bar_eids = POSTMOD_BAR_STRESS_DF.EID.unique()

premod_result = []

postmod_result = []

#         premod_result.append((eid, get_premod_bar_element_life(eid = eid, template_df = TEMPLATE_DF, stress_df = PREMOD_BAR_STRESS_DF)))

pre_bar_summary_df = pd.DataFrame({'A' : []})

post_bar_summary_df = pd.DataFrame({'A' : []})

for eid in premod_bar_eids:
    pre_template['eid'] = eid
    try:
        df = custom_manipulations(**pre_template)

        if pre_bar_summary_df.empty == True:
            pre_bar_summary_df = df.copy()
        else:
            pre_bar_summary_df = pd.concat([pre_bar_summary_df, df], axis=0, sort=False)

    except IndexError:
        print('premod failed at EID: {}'.format(eid))
        
        
for eid in postmod_bar_eids:
    post_template['eid'] = eid
    try:
        df = custom_manipulations(**post_template)
        
        if post_bar_summary_df.empty == True:
            post_bar_summary_df = df.copy()
        else:
            post_bar_summary_df = pd.concat([post_bar_summary_df, df], axis=0, sort=False)                                            
        
    except IndexError:
        print('postmod failed at EID: {}'.format(eid))
        

Found: Fatigue_output.out 
 Action: Deleted


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in true_divide


In [32]:
pre_bar_summary_df.to_excel('pre_rest.xlsx')

In [31]:
premod_bar_eids

(6270,)

In [ ]:
df1 = pd.DataFrame(premod_result, columns=['EID', 'premod_Life'])
# df1.head()

df2 = pd.DataFrame(postmod_result, columns=['EID', 'postmod_Life'])
# df2.head()

result_df = df1.merge(df2, how='outer', on=['EID'])

# result_df.head()

result_df['knockdown_factor'] = result_df.postmod_Life / result_df.premod_Life
# result_df.head()

# Write to excel

In [ ]:
with pd.ExcelWriter('Fatigue_output.xlsx') as writer:  # doctest: +SKIP
    
#     premod_spectrum_df.to_excel(writer, sheet_name='premod_spectrum')
    
#     premod_rain_table.to_excel(writer, sheet_name='premod_S-N')
    
#     postmod_spectrum_df.to_excel(writer, sheet_name='postmod_spectrum')
    
#     postmod_rain_table.to_excel(writer, sheet_name='postmod_S-N')
    
    result_df.to_excel(writer, sheet_name='Life_estimation')

In [ ]:
print('ALL DONE WELL!')

In [ ]:
pre_bar_summary_df.set_index('EID', inplace=True)

pre_bar_summary_df = pre_bar_summary_df.add_prefix('PRE-MOD_')

post_bar_summary_df.set_index('EID', inplace=True)

post_bar_summary_df = post_bar_summary_df.add_prefix('POST-MOD_')



In [ ]:
bar_result = pd.concat([pre_bar_summary_df, post_bar_summary_df], axis=1)

In [40]:
post_bar_summary_df.head()

,EID,Max,Min,max_line_Num,min_line_Num,R,Kt_const,Smax_const,S_Max,Kt_sigma,Seq,Nf,_1_Over_Nf,One_Over_Nf_sum_of_1_over_Nf,Life,pct_damage
Index,,,,,,,,,,,,,,,,
6,1110301,6.245070e+00,-1.459337e+01,24,4,-2.336783,3.0,6.895,9.057389e-01,2.717217e+00,5.084577e+00,2.571245e+14,3.889166e-15,1.000000,2.571245e+14,100.0%
7,1110302,7.601613e+00,-4.470740e+00,24,8,-0.588130,3.0,6.895,1.102482e+00,3.307446e+00,4.206817e+00,1.439667e+15,6.946052e-16,1.000000,1.439667e+15,100.0%
7,1110303,3.715907e+00,-9.513014e-01,24,8,-0.256008,3.0,6.895,5.389278e-01,1.616783e+00,1.820237e+00,2.920371e+18,3.424223e-19,1.000000,2.920371e+18,100.0%
6,1110304,5.094818e+00,-5.769386e+00,24,6,-1.132403,3.0,6.895,7.389148e-01,2.216745e+00,3.286455e+00,1.358143e+16,7.362993e-17,0.999995,1.358137e+16,100.0%
7,1110305,1.156750e-12,-2.891875e-13,22,12,-0.250000,3.0,6.895,1.677665e-13,5.032995e-13,5.652228e-13,1.453192e+132,6.881405e-133,0.589408,8.565227e+131,58.9%


In [43]:
post_bar_summary_df.shape[0]

5898

In [42]:
temp_pre = pre_bar_summary_df.copy()

In [ ]:
bar_result = pd.concat([pre_bar_summary_df, post_bar_summary_df], axis=1, ignore_index=True)

In [44]:
idx = np.arange(post_bar_summary_df.shape[0])
len(idx)

5898

In [46]:
temp_pre = temp_pre.reset_index()
temp_pre.head()

,Index,EID,Max,Min,max_line_Num,min_line_Num,R,Kt_const,Smax_const,S_Max,Kt_sigma,Seq,Nf,_1_Over_Nf,One_Over_Nf_sum_of_1_over_Nf,Life,pct_damage
0,6,1110301,6.335253,-14.888807,24,4,-2.350152,3.0,6.895,0.918818,2.756455,5.168738,2.214802e+14,4.515076e-15,1.000000,2.214802e+14,100.0%
1,7,1110302,7.729535,-4.623518,24,8,-0.598163,3.0,6.895,1.121035,3.363104,4.291640,1.200756e+15,8.328087e-16,1.000000,1.200756e+15,100.0%
2,7,1110303,3.785034,-0.991872,24,8,-0.262051,3.0,6.895,0.548953,1.646860,1.858732,2.414456e+18,4.141720e-19,1.000000,2.414456e+18,100.0%
3,6,1110304,5.131298,-5.747056,24,6,-1.120001,3.0,6.895,0.744206,2.232617,3.299962,1.308441e+16,7.642682e-17,0.999996,1.308436e+16,100.0%
4,6,1110306,2.989302,-16.547781,24,4,-5.535667,3.0,6.895,0.433546,1.300639,3.452292,8.681650e+15,1.151855e-16,1.000000,8.681650e+15,100.0%


In [47]:
temp_pre.EID.value_counts()

1604007    2
1110313    2
1703107    2
1703065    2
4212108    1
1110707    1
1141501    1
4242102    1
1120304    1
4115105    1
1123001    1
1434004    1
4216102    1
1123005    1
4248107    1
1129134    1
1141211    1
1127103    1
1123009    1
1124603    1
1127107    1
1436015    1
1442501    1
1127111    1
5305007    1
1129130    1
1422004    1
1514186    1
1424014    1
1614008    1
          ..
1121713    1
1134003    1
3313014    1
1515135    1
1134007    1
1906104    1
1140126    1
1136028    1
1437015    1
1519003    1
5106813    1
1801601    1
1435010    1
1419026    1
1435014    1
1136008    1
1605001    1
1140106    1
1136012    1
1141032    1
1435022    1
1136016    1
4241101    1
1140114    1
1136020    1
1703043    1
1435030    1
1136024    1
1140122    1
1114112    1
Name: EID, Length: 5998, dtype: int64

In [50]:
temp_pre.loc[temp_pre.EID == 1703065]

,Index,EID,Max,Min,max_line_Num,min_line_Num,R,Kt_const,Smax_const,S_Max,Kt_sigma,Seq,Nf,_1_Over_Nf,One_Over_Nf_sum_of_1_over_Nf,Life,pct_damage
4062,8,1703065,2.273737e-14,-9.094947e-14,16,20,-4.0,3.0,6.895,3.297660e-15,9.892981e-15,2.284502e-14,6.738832e+144,1.483937e-145,0.5,3.369416e+144,50.0%
4063,9,1703065,2.273737e-14,-9.094947e-14,16,20,-4.0,3.0,6.895,3.297660e-15,9.892981e-15,2.284502e-14,6.738832e+144,1.483937e-145,0.5,3.369416e+144,50.0%


In [ ]:
premod